In [16]:
'''
Author  : Jiayao Yu, User Interfaces Group, Aalto University
Init    : August, 2017
Project : ELEC-D7010 Engineering for Humans course materials
Topic   : Multiple Resource Theory
'''

'''
Usage explaination
'''

import tkinter as tk
from tkinter import ttk
import numpy as np
import matplotlib
# matplotlib.use("TkAgg")                         # for 3d cube visualization
# import matplotlib.backends.backend_tkagg as tkagg
from IPython import display
%matplotlib inline
import matplotlib.pyplot as plt

class MRT(tk.Frame):
	tasks = ['driving', 'reading-map', 'listening-navigation']
	indexSelectedTask = []		# index of 2 selected tasks

	def __init__(self, parent=None):
		tk.Frame.__init__(self, parent)
		self.pack()
		self.makeWidgets()

	def makeWidgets(self):
		self.frameTaskSelection = tk.Frame(self)
		self.frameTaskAnalysis = tk.Frame(self)
		self.frameConflictMatrix = tk.Frame(self)
		self.frameCubeVis = tk.Frame(self)
		self.frameTaskSelection.grid(row=0, column=0, sticky=tk.NSEW)
		self.frameTaskAnalysis.grid(row=1, column=0, sticky=tk.NSEW)
		self.frameConflictMatrix.grid(row=2, column=0, sticky=tk.NSEW)
		self.frameCubeVis.grid(row=3, column=0, sticky=tk.NSEW)
		self.btnsTask = []
		for i in range(len(self.tasks)):
			n = tk.Button(self.frameTaskSelection, text=self.tasks[i], width=20)
			self.btnsTask.append(n)
			self.btnsTask[-1].grid(row=i, column=0, sticky=tk.NSEW)
			self.btnsTask[-1].bind('<ButtonPress-1>', self.taskSelection)
	
	def taskSelection(self, event):
		for i in range(len(self.tasks)):
			if event.widget is self.btnsTask[i]:
				if self.btnsTask[i].cget('bg') != 'green':
					if len(self.indexSelectedTask) <= 2:
						self.indexSelectedTask.append(i)
						self.btnsTask[i].config(bg='green', fg='white', relief=tk.SUNKEN)
				else:
					for i in range(2):
						if self.indexSelectedTask[i] == i:
							del self.indexSelectedTask[i] 
					self.btnsTask[i].config(bg='SystemButtonFace', fg='black', relief=tk.RAISED)
		print(self.indexSelectedTask)

	def taskAnalysis(self):
		self.tableTaskAnalysisShell = ttk.Treeview(self.frameTaskAnalysis, columns=('codes', 'stages', 'modalities'), selectmode = "browse")
		self.labelTaskAnalysisTitle = tk.Label(self.frameTaskAnalysis, text="Task Analysis Shell", anchor=tk.CENTER)
		self.tableTaskAnalysisShell.grid(row=0, column=0)
		self.labelTaskAnalysisTitle.grid(row=1, column=0)
		self.tableTaskAnalysisShell.column('#0', width=100)
		self.tableTaskAnalysisShell.column('codes', width=70, stretch=True)
		self.tableTaskAnalysisShell.column('stages', width=70, stretch=True)
		self.tableTaskAnalysisShell.column('modalities', width=70, stretch=True)
		self.tableTaskAnalysisShell.heading('codes', text='Codes')
		self.tableTaskAnalysisShell.heading('stages', text='Stages')
		self.tableTaskAnalysisShell.heading('modalities', text='Modalities')
		self.tableTaskAnalysisShell.insert('', 0, 'driving', text=self.tasks[0], tags=0, values='S PC+R V')
		self.tableTaskAnalysisShell.insert('', 1, 'reading', text=self.tasks[1], tags=1, values='S PC+R V')
		self.tableTaskAnalysisShell.insert('', 2, 'listening', text=self.tasks[2], tags=2, values='V PC A')
		self.taskAnalysisValues = [[[1, 0], [1, 1], [0, 1]], [[1, 0], [1, 1], [0, 1]], [[0, 1], [1, 0], [1, 0]]]     # in terms of taskAnalysisValues[task][dimension][level]

	def cubeVis(self):
		pass

	def interference(self):
		pass


if __name__ == "__main__":
	root = tk.Tk()
	root.title("Multiple Resource Theory")
	MRT(root).mainloop()
	root.after(5, lambda: root.focus_force())

[0]
[0, 1]
[0, 1, 2]
[1, 2]
[1, 2]
[1, 2]
[1, 2]
[1, 2]
[1, 2]
[1, 2, 1]
[1, 2, 1]
[1, 2, 1]
[1, 2, 1]
[1, 2, 1]
[1, 2, 1]
[1, 2, 1]
